In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 목차 timestamp 제거 

using_cols = ['user_id', 'movie_id', 'rating']
ratings = ratings[using_cols]
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count1'}, inplace=True)

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
#데이터를 병합합니다.
df = movies.set_index("movie_id")

ratings = ratings.join(df, on = "movie_id")

ratings

,user_id,movie_id,count1,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...
1000203,6040,1090,3,Platoon (1986),Drama|War
1000205,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,Sophie's Choice (1982),Drama


2) 분석해 봅시다.   
- ratings에 있는 유니크한 영화 개수   
- rating에 있는 유니크한 사용자 수   
- 가장 인기있는 영화 30개(인기순)   
   
3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.   
4) CSR matrix를 직접 만들어 봅시다.   
5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.   
6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.   
7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.   
8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.   

# 2) 데이터 분석

In [7]:
# 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# 유니크한 사용자 수 
ratings['user_id'].nunique()

6039

In [9]:
# 인기 많은 영화
rate_count = ratings.groupby('title')['user_id'].count()
rate_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [10]:
# 유저별 몇 개의 영화를 보고 있는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

# 3) 내가 선호하는 영화 5가지 rating에 추가

In [11]:
#'Toy Story 2 (1999)' , 'Jumanji (1995)' ,'Men in Black (1997)' ,'Matrix, The (1999)' ,'Titanic (1997)'
my_favorite = ['3114' , '2' ,'1580' ,'2571' ,'1721']

# 'on'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_favorite, 'count1':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'on'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count1,title,genre
1000203,6040,1090,3,Platoon (1986),Drama|War
1000205,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,Sophie's Choice (1982),Drama
1000208,6040,1097,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,6041,3114,5,NaN,NaN
1,6041,2,5,NaN,NaN
2,6041,1580,5,NaN,NaN
3,6041,2571,5,NaN,NaN
4,6041,1721,5,NaN,NaN


# 4) CSR matrix

In [12]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [13]:
print(user_to_idx[6041])
print(movie_to_idx[3114]) 

6039
50


In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')
    
ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count1,title,genre
0,0,0,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,0,1,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,0,2,3,My Fair Lady (1964),Musical|Romance
3,0,3,4,Erin Brockovich (2000),Drama
4,0,4,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...
0,6039,3628,5,NaN,NaN
1,6039,3629,5,NaN,NaN
2,6039,3630,5,NaN,NaN
3,6039,3631,5,NaN,NaN


In [15]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.count1, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델 

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

In [19]:
me, Toystory = user_to_idx[6041], movie_to_idx[3114]
me_vector, Toystory_vector = als_model.user_factors[me], als_model.item_factors[Toystory]


In [20]:
me_vector

array([-0.00287946,  0.00481963,  0.00224159, -0.00710306,  0.00017792,
        0.01829157,  0.01262657,  0.00856861, -0.00165586, -0.00884277,
        0.0070715 , -0.01798456, -0.00913327, -0.00243028, -0.01030118,
        0.00429819, -0.00413618,  0.01655273,  0.01085787, -0.021388  ,
        0.01053767, -0.01689044, -0.00488323,  0.0135571 ,  0.01212728,
        0.00922782, -0.02042558,  0.02749887,  0.00491108, -0.00540041,
        0.00982979,  0.0402177 , -0.00172826, -0.03216823, -0.00938061,
       -0.00321913, -0.00352263, -0.01513936,  0.00781351, -0.00880751,
       -0.00766609,  0.00431863,  0.00432134, -0.01250757,  0.00609524,
       -0.016424  ,  0.00769079,  0.00921687, -0.01793297,  0.00289883,
       -0.00674938,  0.01764936,  0.01548396,  0.01530269,  0.00191433,
       -0.00976498,  0.01679956,  0.00381855,  0.0092501 , -0.00197568,
        0.01576417,  0.01828542,  0.04162085, -0.01494062, -0.01032756,
       -0.02580396,  0.00351279, -0.00423645,  0.02365849, -0.01

In [21]:
Toystory_vector

array([ 8.30566697e-03,  1.77543890e-02,  3.71281244e-02,  1.80215202e-02,
        3.60637978e-02, -1.43960584e-02, -1.82511713e-02, -1.07911602e-02,
       -1.85396802e-02,  2.91437246e-02,  3.18335695e-03,  4.21694629e-02,
        1.35511644e-02,  4.28009965e-02, -4.47298493e-03,  4.54336929e-04,
       -3.32756378e-02,  9.47239436e-03,  2.03162106e-03,  5.19822445e-03,
        4.71809506e-02,  5.14095975e-03, -1.47986319e-02,  6.35832129e-03,
       -2.85184458e-02, -6.80223061e-03, -1.76405050e-02, -4.83822683e-03,
       -1.37891844e-02, -7.60680472e-04,  1.83079299e-02,  3.44035327e-02,
       -1.11928340e-02, -7.11211981e-03, -1.46685410e-02, -4.68023261e-03,
       -1.96468830e-02, -1.63349509e-02,  1.56426113e-02,  2.26246119e-02,
        2.51919739e-02,  1.27956376e-03,  1.19087202e-02, -4.32622135e-02,
        1.49003221e-02,  1.61575135e-02,  3.13565577e-03,  2.34283525e-02,
       -3.74706718e-03,  1.80128366e-02, -2.73292717e-02,  7.76227145e-03,
       -3.59882526e-02,  

In [24]:
# me과 Toystory2를 내적하는 코드
np.dot(me_vector, Toystory_vector)

0.0025182352

# 6) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [30]:
idx_to_movie = {}

for idx, movie in enumerate(movies['movie_id']):
    idx_to_movie[movie] = movies['title'][idx]


favorite_movie_id = 3114 # Toy Story 2 (1999)
similar_movie = als_model.similar_items(favorite_movie_id, N=15)

for m in similar_movie:
    m = m + (idx_to_movie[m[0]],)
    print(m)
# similar_movie

(3114, 0.02170068, 'Toy Story 2 (1999)')
(2346, 0.014962188, 'Stepford Wives, The (1975)')
(1199, 0.0147929005, 'Brazil (1985)')
(1941, 0.014673396, 'Hamlet (1948)')
(3197, 0.014647422, 'Presidio, The (1988)')
(1963, 0.014467618, 'Take the Money and Run (1969)')
(2904, 0.014328996, 'Rain (1932)')
(1970, 0.014317073, 'Nightmare on Elm Street 3: Dream Warriors, A (1987)')
(1672, 0.014250074, 'Rainmaker, The (1997)')
(3062, 0.014208445, 'Longest Day, The (1962)')
(1962, 0.014118617, 'Driving Miss Daisy (1989)')
(1973, 0.013932802, "Freddy's Dead: The Final Nightmare (1991)")
(2134, 0.013920384, 'Weird Science (1985)')
(1975, 0.013817077, 'Friday the 13th Part 2 (1981)')
(1974, 0.013750525, 'Friday the 13th (1980)')


# 7) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.  

In [42]:
user = user_to_idx[6041]
movie_recommended = als_model.recommend(user, csr_data, N=5, filter_already_liked_items=True)

for i in movie_recommended:
    i = i + (idx_to_movie[i[0]], )
    print(i)

# movie_recommended

(107, 0.0048022233, 'Muppet Treasure Island (1996)')
(2229, 0.0044038584, 'Pleasure Garden, The (1925)')
(41, 0.0040777894, 'Richard III (1995)')
(232, 0.004071654, 'Eat Drink Man Woman (1994)')
(258, 0.0040616645, "Kid in King Arthur's Court, A (1995)")
